In this app, we let the estimated GDP-CO2 relationship vary over time by including time as an additional input into the neural network. 

In [76]:
%matplotlib inline

import pandas as pd
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import seaborn as sns; sns.set()

from ipywidgets import interact, fixed
import ipywidgets as widgets

In [107]:
# Creating plotting function
def SurfPlot(region, loss, architecture):
    """
    Makes 3D surface plot for a given region in a given year.

    ARGUMENTS
        * region:       Must be from the list: 'World', 'OECD', 'Asia', 'REF', 'MAF', 'LAM'.
        * loss:         Must be from the list: 'MSE', 'MAE'
        * architecture: Must be from the list: (4), (8), (16), (8,4), (16,8,4) 
    """
       
    # Loading data
    time        = np.load('Models/{}/Misc/time.npy'.format(region), allow_pickle=True)
    T           = np.load('Models/{}/Misc/T.npy'.format(region), allow_pickle=True)
    Min         = np.load('Models/{}/Misc/min.npy'.format(region), allow_pickle=True)
    Max         = np.load('Models/{}/Misc/max.npy'.format(region), allow_pickle=True)
    individuals = np.load('Models/{}/Misc/individuals.npy'.format(region), allow_pickle=True)
    alphas      = np.load('Models/{}/{}/{}/alphas.npy'.format(region, loss, architecture), allow_pickle=True)
    y_train_agg = np.load('Models/{}/Misc/y_train_agg.npy'.format(region), allow_pickle=True)
    
    lGDP = pd.read_pickle('Models/{}/Misc/lGDP'.format(region))[individuals]
    lCO2 = pd.read_pickle('Models/{}/Misc/lCO2'.format(region))[individuals]
    
    in_sample_agg = pd.read_pickle('Models/{}/{}/{}/in_sample_agg'.format(region, loss, architecture))
    model_pred    = tf.keras.models.load_model('Models/{}/{}/{}/model_pred'.format(region, loss, architecture))

    # Surface plot
    ax1      = time - 1959
    ax2      = np.linspace(Min, Max, 1000)

    ax1, ax2 = np.meshgrid(ax1, ax2)

    ax1_vec  = np.reshape(ax1, (-1,1), order='F')
    ax2_vec  = np.reshape(ax2, (-1,1), order='F')
    
    x_input  = np.hstack((ax1_vec, ax2_vec))

    pred     = model_pred(x_input)
    pred     = np.reshape(np.array(pred), (1000, T), order='F')
    
    # Scatter plot
    ax11 = np.vstack([time] * lGDP.shape[1])
    ax22 = np.array(lGDP.loc[time])
    ax33 = np.array(lCO2.loc[time] - np.hstack((np.zeros((1, 1)), alphas)))

    # Figure
    plt.close(None)
    
    fig = plt.figure(figsize=plt.figaspect(0.25))

    ax = fig.add_subplot(1, 3, 1, projection='3d')
    surf = ax.plot_surface(ax1 + 1959, ax2, pred, linewidth=0, antialiased=True, color='black')
    ax.set_xlabel('year', fontweight='bold')
    ax.set_ylabel('log(GDP)', fontweight='bold')
    ax.set_zlabel('log(CO$_2$)', fontweight='bold')
    ax.set_zlim(-3,5)
    ax.view_init(36,-140)
    ax.zaxis.labelpad=-0
    plt.title('Estimated surface', fontweight='bold')
    
    pred = np.mean(pred,axis=1)
    
    ax = fig.add_subplot(1, 3, 2)
    plot = ax.plot(np.linspace(Min, Max, 1000), pred, antialiased=True, color='black')
    scat = ax.scatter(ax22, ax33, s=2, label='log(CO$_2$) minus estimated country fixed effects')
    ax.set_xlabel('log(GDP)', fontweight='bold')
    ax.set_ylabel('log(CO$_2$)', fontweight='bold')
    ax.yaxis.labelpad=-2
    ax.set_ylim(-3,5)
    plt.title('Time-series average of estimated surface', fontweight='bold')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), fancybox=True, shadow=True, ncol=1)
    
    ax = fig.add_subplot(1, 3, 3)
    axis = np.reshape(np.array(time), (-1,1), order='F')
        
    if region=='REF':
        ax.plot(axis, in_sample_agg[5:], color='black', label='Model predictions')
        ax.plot(axis, y_train_agg[5:], color='black', linestyle='dashdot', label='Historical emissions')
    else:    
        ax.plot(axis, in_sample_agg, color='black', label='Model predictions')
        ax.plot(axis, y_train_agg, color='black', linestyle='dashdot', label='Historical emissions')
    ax.set_ylabel('Mt CO$_2$', fontweight='bold')
    ax.set_xlabel('year', fontweight='bold')
    ax.yaxis.labelpad=-2
    if region=='World':
        ax.set_ylim(2000,45000)
    elif region=='OECD':
        ax.set_ylim(2000,15000)
    elif region=='REF':
        ax.set_ylim(0,4500)
    elif region=='Asia':
        ax.set_ylim(0,25000)
    elif region=='MAF':
        ax.set_ylim(0,4000)
    elif region=='LAM':
        ax.set_ylim(0,2000)    
    plt.title('Aggregate predictions', fontweight='bold')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), fancybox=True, shadow=True, ncol=1)
    
    plt.show()
    

#### Estimated Surface

In [108]:
interact(SurfPlot, region=['World', 'OECD', 'REF', 'Asia', 'MAF', 'LAM'], loss=['MSE', 'MAE'], architecture=['(4)', '(8)', '(16)', '(8,4)', '(16,8,4)']);

interactive(children=(Dropdown(description='region', options=('World', 'OECD', 'REF', 'Asia', 'MAF', 'LAM'), v…